# 文本分析案例: 停用词和关键词提取

1. 进行文本分析时, 需要先去掉那些**常见的, 但是没有什么意义**的词, 例如"的", "啊"

2. 词频TF = 某个词在该文章中的频率

3. 逆文档频率

    $$IDF = log(样本总数 / (1 + 出现该关键词的文档数) )$$

$$TF-IDF指标 = TF * IDF$$

找出一篇文章中出现次数高的关键词, 从这些关键词中再找出独属于该篇文章特点的**专属关键词**

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import jieba

In [2]:
df = pd.read_xml('../../data/input/news_xml.xml')
df.head()

,url,docno,contenttitle,content
0,http://news.ifeng.com/mil/video/detail_2012_07...,215963ca2a4f24c1-bf8e056905f04e60,队友失常阿尔沙文无奈独舞 沙皇恐就此告别欧洲杯,凤凰网体育讯 北京时间６月１７日，此前的出线热门俄罗斯队在大平即可出线的大好形势下被希腊队上...
1,http://news.ifeng.com/mil/video/detail_2012_07...,f62decca2a4f24c1-bf8e056905f04e60,日本欲借海峡当天险 却终被美军潜艇突破,日本欲借海峡当天险 却终被美军潜艇突破
2,http://news.ifeng.com/mainland/detail_2012_07/...,51ff1dcb2a4f24c1-bf8e056905f04e60,内地将建统一社保信息平台,新华社电 国务院总理温家宝４日主持召开国务院常务会议，部署２０１１年度中央预算执行等审计查出...
3,http://news.ifeng.com/society/1/detail_2012_07...,79293ccb2a4f24c1-bf8e056905f04e60,江苏一男子劫持银行工作人员 僵持４０分钟被制服,中新网盐城７月１１日电（谷华）１１日上午，江苏盐城一男子在银行门口持刀劫持银行职员，僵持４０...
4,http://news.ifeng.com/sports/gjzq/xj/basa/deta...,189d4dcb2a4f24c1-bf8e056905f04e60,巴萨女足签美艳新援 西媒惊呼可当模特（图）,巴萨男足在今夏的引援市场上并不惊艳，但是女足球队引进了一名模特级美女新援则引发了西媒的强烈关...


In [4]:
df.columns

Index(['url', 'docno', 'contenttitle', 'content'], dtype='object')

In [3]:
df['content'].head()

0    凤凰网体育讯　北京时间６月１７日，此前的出线热门俄罗斯队在大平即可出线的大好形势下被希腊队上...
1                                  日本欲借海峡当天险　却终被美军潜艇突破
2    新华社电　国务院总理温家宝４日主持召开国务院常务会议，部署２０１１年度中央预算执行等审计查出...
3    中新网盐城７月１１日电（谷华）１１日上午，江苏盐城一男子在银行门口持刀劫持银行职员，僵持４０...
4    巴萨男足在今夏的引援市场上并不惊艳，但是女足球队引进了一名模特级美女新援则引发了西媒的强烈关...
Name: content, dtype: object

# 分词器使用

In [9]:
contents = df['content'].tolist()
contents

['凤凰网体育讯\u3000北京时间６月１７日，此前的出线热门俄罗斯队在大平即可出线的大好形势下被希腊队上演神话，０：１负于对手，送希腊队出线的时候，也把自己送回了家。本场比赛俄罗斯队核心阿尔沙文的表现中规中矩，为队友创造了几次机会，可惜队友都没有把握住。\ue40c前两轮小组赛，俄罗斯４－１大胜捷克，１－１战平波兰，积４分占据Ａ组榜首位置。对阵希腊，俄罗斯只需要１场平局即可出线。如果能够击败希腊，那么俄罗斯铁定以小组头名晋级八强。显然，俄罗斯希望拿到小组第１，从而在８强战中回避Ｂ组头名（很有可能就是德国）。而希腊队虽然目前积分垫底，但如果他们能在最后一轮击败俄罗斯，\u3000同样可以出线。\ue40c希腊队虽然早已走下神坛，但实力尚存，依然让许多球队所忌惮。本届欧洲杯上，他们差点就搞砸了东道主波兰的首秀。对俄罗斯而言，这是一场不容有失的比赛。而队中的中场核心阿尔沙文的表现，很大程度上决定了俄罗斯队的命运。首战对捷克，沙皇表现出色，俄罗斯４－１大胜对手。次战俄罗斯被波兰逼平，阿尔沙文的表现受到了主教练艾德沃卡特的严厉批评：＂我们上一场踢得不好，阿尔沙文丢球太多，而且下半场的时候，他显然是体力透支了。但他是球队队长，也是我们最好的攻击手，是那种能够凭借一己之力改变比赛结果的球员，所以我别无选择，只能把他留在场上。＂\ue40c开场仅２分钟，阿尔沙文就策动进攻，可惜他在禁区前沿的传球被对方门将得到。第１０分钟，俄罗斯队右路的进攻，传中球给到中路，阿尔沙文在对手紧逼下的头球攻门顶的太正被希腊门将西法基斯拿到。第２０分钟，阿尔沙文禁区前远射打偏。此后，俄罗斯队逐渐控制了比赛的节奏。第３０分钟，阿尔沙文禁区前直塞，球被对方后卫抢先解围。第３８分钟，阿尔沙文左路４５度起球没有传起来第一点被对手破坏出来。第４５分钟，阿尔沙文禁区内传球，俄罗斯队员的两次射门均被对手挡出。\ue40c下半场俄罗斯队大举反攻，但效果一般。第６３分钟，俄罗斯队获得任意球机会，任意球快速开出，阿尔沙文右路挑传球被后卫顶出来。第７４分钟，阿尔沙文直塞球给日尔科夫，日尔科夫倒地。第７６分钟，阿尔沙文禁区内转身摆脱直塞被希腊队铲出。第８４分钟，俄罗斯队右路的进攻，阿尔沙文右路传中，扎戈耶夫头球攻门擦着立柱出了底线。第９２分钟，阿尔沙文回到后场左路拿球传给无人防守日尔科夫，日尔科夫的传中球给后点太大，裁判先吹了

In [15]:
words: set[str] = set()
for content in contents:
    if content is not None:
        content = content.replace("\u3000", ":")
        lcut = jieba.lcut(content)
        words.update(lcut)
words

{'扮演',
 '筒',
 '东盟',
 '红旗',
 '晚收',
 '新',
 '原本',
 '另外',
 '接应',
 '吉安市',
 '世锦赛',
 '整形',
 '主场',
 '统计',
 '拨通',
 '抢眼',
 '参赞',
 '深造',
 '中俄关系',
 '矛盾',
 '次脚误',
 '研讨会',
 '这艘',
 '地库',
 '但语',
 '上演',
 '点半',
 '天下',
 '销售',
 '两分',
 '同',
 '一如',
 '表演赛',
 '进球',
 '小天',
 '寻觅',
 '主张',
 '这才',
 '昨晚',
 '金英权',
 '中华民族',
 '配备',
 '擅长',
 '并',
 '处级',
 '忧虑',
 '告知',
 '子孙',
 '刑事',
 '足球场',
 '万美元',
 '亚银',
 '偏见',
 '虱子',
 '太',
 '摁',
 '入轨',
 '岁月',
 '分差',
 '人和队',
 '登场',
 '并非',
 '呗',
 '陈亭',
 '社会保险',
 '打到',
 '竞技场',
 '不同',
 '格雷',
 '多时',
 '播放器',
 '焉能',
 '一台',
 '何去何从',
 '全省各地',
 '力争',
 '半岛',
 '精神',
 '必称',
 '俄超',
 '不用',
 '陪同',
 '奥斯卡',
 '全套',
 '高原',
 '首套',
 '最快',
 '大学',
 '争夺',
 '推特上',
 '两万平方米',
 '交界',
 '多番',
 '度',
 '图名',
 '被判',
 '四成',
 '底定',
 '祝贺',
 '请求',
 '大',
 '密报',
 '当天',
 '不变',
 '方向盘',
 '大使',
 '无缘',
 '北大',
 '董会峰',
 '仔',
 '班里',
 '克省',
 '传给',
 '甚孝',
 '湖人时',
 '此事',
 '起',
 '只要',
 '一楼',
 '十分钟',
 '裁决',
 '作案',
 '花钱',
 '女生',
 '众所周知',
 '外媒',
 '老出',
 '关系',
 '中队',
 '刀',
 '涉案',
 '休战',
 '隐秘',
 '之列',
 '今夏',
 '聘书',
 '第二架',
 '先进',
 '横空出世',
 '交通

# 过滤停用词